In [2]:
from deepface import DeepFace
import re


In [3]:
def authorise_user(image_path):
    print('\n-------------- Authorisation --------------\n')

    # anti spoof
    # img = Image.open(image_path)

    # input = preprocess(img).unsqueeze(0)
    # cls_out, feature = model(input, norm_flag=True)
    # prob = F.softmax(cls_out, dim=1).cpu().data.numpy()

    # spoof_result = ((prob[:, 1])>= 0.5).astype(int)[0]

    face_objs = DeepFace.extract_faces(
    img_path=image_path,
    anti_spoofing = True
    )

    if not all(face_obj["is_real"] is True for face_obj in face_objs):
        return 'Authorisation DENIED, spoof detected'

    else:
        result = DeepFace.find(image_path, db_path='./database')

        distance = result[0]['distance'][0]
        person = ' '.join((result[0]['identity'][0]).split('\\')[-1].split('_')[:2])
        max_distance = 0.5

        print(f'Distance: {distance}')
        if distance <= max_distance:
            print(f'Authorisation confirmed. Welcome {person}!')
        else:
            print('Authorisation DENIED')
            
        return result

In [6]:
result = authorise_user('dataset/Adam_Sandler/test_positive/Adam_Sandler_1088_560.jpeg')



-------------- Authorisation --------------

24-06-16 16:48:07 - Searching dataset/Adam_Sandler/test_positive/Adam_Sandler_1088_560.jpeg in 7101 length datastore
24-06-16 16:48:09 - find function duration 8.096940994262695 seconds
Distance: 0.0927406750075257
Authorisation confirmed. Welcome Adam Sandler!


In [ ]:
from deepface import DeepFace
face_objs = DeepFace.extract_faces(
  img_path="test_6000/10001/spoof/10001_12.png",
  anti_spoofing = True
)

In [ ]:
# assert all(face_obj["is_real"] is True for face_obj in face_objs)

In [10]:
# for face_obj in face_objs:
#     print(face_obj["is_real"])

if not all(face_obj["is_real"] is True for face_obj in face_objs):
    print('spoof')
else:
    print('no spoof')


spoof


TESTOWANIE


In [32]:
import os
import shutil
import random
from collections import defaultdict

def load_and_save_image_paths(directory, target_directory, categories, limits):
    total_images = 0
    category_files = defaultdict(list)
    global_limit = sum(limits.values())  # Suma wszystkich limitów

    for person in os.listdir(directory):
        person_path = os.path.join(directory, person)
        if os.path.isdir(person_path):
            for category in categories:
                category_path = os.path.join(person_path, category)
                if os.path.exists(category_path):
                    for root, dirs, files in os.walk(category_path):
                        for file in files:
                            if file.lower().endswith(('.png', '.jpeg', '.jpg')):
                                full_path = os.path.join(root, file)
                                category_files[category].append(full_path)

    # Losowe wybieranie plików z ograniczeniami i zapisywanie
    for category, files in category_files.items():
        if len(files) > limits[category]:
            files = random.sample(files, limits[category])
        for full_path in files:
            if total_images >= global_limit:
                break  # Przerywa, jeśli osiągnięto globalny limit
            file = os.path.basename(full_path)
            relative_path_parts = full_path.split(os.sep)[1:-1]
            sub_folder = os.path.join(target_directory, *relative_path_parts)
            os.makedirs(sub_folder, exist_ok=True)
            target_path = os.path.join(sub_folder, file)
            shutil.copy(full_path, target_path)
            print(f"Copied {file} to {target_path}")
            total_images += 1

    print(f"Total images processed: {total_images}")
    return total_images

os.makedirs('test_6000', exist_ok=True)
db_path = './database'
categories = ['test_positive', 'spoof', 'test_low_res']
limits = {'test_positive': 1000, 'spoof': 2500, 'test_low_res': 2500}
total_images_loaded_and_saved = load_and_save_image_paths('dataset', 'test_6000', categories, limits)



Copied 5768_61.png to test_6000\5768\spoof\5768_61.png
Copied 10007_3.png to test_6000\10007\spoof\10007_3.png
Copied 5916_116.png to test_6000\5916\spoof\5916_116.png
Copied 4978_16.png to test_6000\4978\spoof\4978_16.png
Copied 4931_19.png to test_6000\4931\spoof\4931_19.png
Copied 5774_2.png to test_6000\5774\spoof\5774_2.png
Copied 10068_33.png to test_6000\10068\spoof\10068_33.png
Copied 4943_4.png to test_6000\4943\spoof\4943_4.png
Copied 10138_41.png to test_6000\10138\spoof\10138_41.png
Copied 4943_187.png to test_6000\4943\spoof\4943_187.png
Copied 4931_91.png to test_6000\4931\spoof\4931_91.png
Copied 10109_20.png to test_6000\10109\spoof\10109_20.png
Copied 10054_11.png to test_6000\10054\spoof\10054_11.png
Copied 4931_59.png to test_6000\4931\spoof\4931_59.png
Copied 5723_165.png to test_6000\5723\spoof\5723_165.png
Copied 10025_26.png to test_6000\10025\spoof\10025_26.png
Copied 10007_22.png to test_6000\10007\spoof\10007_22.png
Copied 5023_154.png to test_6000\5023\spoof\

In [3]:
import os
from deepface import DeepFace
import pandas as pd

def test_images(directory, db_path):
    total_images = 0
    correct_identifications = 0

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.png', '.jpeg', '.jpg')):
                full_path = os.path.join(root, file)
                total_images += 1

                try:
                    # Directly extract faces without anti-spoofing
                    face_objs = DeepFace.extract_faces(img_path=full_path)

                    # Proceed if faces are detected
                    if face_objs:
                        result = DeepFace.find(img_path=full_path, db_path=db_path)
                        if isinstance(result, pd.DataFrame) and not result.empty:
                            if result.iloc[0]['distance'] <= 0.5:
                                correct_identifications += 1

                except Exception as e:
                    print(f"An error occurred while processing {file}: {str(e)}")

    # Metric Calculations
    TIR = correct_identifications / total_images * 100 if total_images > 0 else 0
    FAR = (total_images - correct_identifications) / total_images * 100 if total_images > 0 else 0
    FRR = (total_images - correct_identifications) / total_images * 100 if total_images > 0 else 0

    print(f"Total images processed: {total_images}")
    print(f"Correctly identified images: {correct_identifications}")
    print(f"True Identification Rate (TIR): {TIR:.2f}%")
    print(f"False Acceptance Rate (FAR): {FAR:.2f}%")
    print(f"False Rejection Rate (FRR): {FRR:.2f}%")

    return TIR, FAR, FRR

# Preparation and testing
db_path = './database'
directory = 'test_6000'
TIR, FAR, FRR = test_images(directory, db_path)


An error occurred while processing 10001_12.png: Face could not be detected in test_6000\10001\spoof\10001_12.png.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
24-06-17 07:08:09 - Searching test_6000\10001\spoof\10001_18.png in 7101 length datastore
24-06-17 07:08:11 - find function duration 5.538081884384155 seconds
An error occurred while processing 10001_22.png: Face could not be detected in test_6000\10001\spoof\10001_22.png.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
24-06-17 07:08:16 - Searching test_6000\10001\spoof\10001_28.png in 7101 length datastore
24-06-17 07:08:20 - find function duration 7.434247016906738 seconds
24-06-17 07:08:31 - Searching test_6000\10001\spoof\10001_31.png in 7101 length datastore
24-06-17 07:08:33 - find function duration 9.692108154296875 seconds
An error occurred while processing 10001_33.png: Face could not be detected in test_6000\1000

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000001E82FA44280>>
Traceback (most recent call last):
  File "c:\Users\Iga Miller\OneDrive\Pulpit\Studia_mgr_SI\Sem3\Biometria\Proj3\.venv\lib\site-packages\ipykernel\ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


24-06-17 07:13:22 - Searching test_6000\10007\spoof\10007_179.png in 7101 length datastore
24-06-17 07:13:27 - find function duration 8.810129642486572 seconds
An error occurred while processing 10007_18.png: Face could not be detected in test_6000\10007\spoof\10007_18.png.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
An error occurred while processing 10007_180.png: Face could not be detected in test_6000\10007\spoof\10007_180.png.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
An error occurred while processing 10007_181.png: Face could not be detected in test_6000\10007\spoof\10007_181.png.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


In [ ]:
import os
import cv2
import ssl
from cv2 import dnn_superres
from deepface import DeepFace
import pandas as pd
import torch
import transforms


# Set up the anti-spoofing model
ssl._create_default_https_context = ssl._create_unverified_context
from src.anti_spoof.fas import flip_it

model = flip_it()
model_path = "pretrained_models/casia_flip_mcl.pth.tar"
checkpoint = torch.load(model_path, map_location="cpu")
model.load_state_dict(checkpoint["state_dict"], strict=False)
model.eval()

# Preprocessing for anti-spoofing model
preprocess = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229])
])

def prepare_super_resolution():
    # Create an SR object
    sr = dnn_superres.DnnSuperResImpl_create()
    
    # Read the desired model
    path = "pretrained_models/EDSR_x3.pb"  # Ensure the correct path to the model
    sr.readModel(path)
    
    # Set the desired model and scale to get correct pre- and post-processing
    sr.setModel("edsr", 3)
    
    return sr

def check_real(img_path):
    """Check if the image is real or a spoof using the anti-spoofing model."""
    img = Image.open(img_path)
    input_tensor = preprocess(img).unsqueeze(0)
    cls_out, feature = model(input_tensor, norm_flag=True)
    prob = F.softmax(cls_out, dim=1).cpu().data.numpy()
    return ((prob[:, 1])>= 0.5).astype(int)[0]  # 0 == spoof, 1 == real

def test_images_spoof(directory, db_path,sr):
    total_images = sum([len(files) for r, d, files in os.walk(directory)])
    passed_antispoofing = 0
    correct_identifications = 0
    spoof_detected_count = 0

    print("Processing images...")
    progress = tqdm(total=total_images, unit='files')

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.png', '.jpeg', '.jpg')):
                full_path = os.path.join(root, file)
                
                # Read and upscale image
                image = cv2.imread(full_path)
                if image is None:
                    print(f"Failed to load {file}")
                    continue
                
                image = sr.upsample(image)
                
                # Save or overwrite the upscaled image for further processing
                cv2.imwrite(full_path, image)
                
                total_images += 1

                try:
                    # Check for anti-spoofing
                    if check_real(full_path) == 1:
                        passed_antispoofing += 1

                        # Proceed with facial recognition
                        result = DeepFace.find(img_path=full_path, db_path=db_path)
                        
                        if isinstance(result, pd.DataFrame) and not result.empty:
                            if result.iloc[0]['distance'] <= 0.5:
                                correct_identifications += 1
                    else:
                        spoof_detected_count += 1

                except Exception as e:
                    print(f"An error occurred while processing {file}: {str(e)}")
                
                progress.update(1)
    progress.close()

    # Calculate metrics
    FRR = (total_images - correct_identifications) / total_images * 100 if total_images > 0 else 0
    FAR = (total_images - passed_antispoofing) / total_images * 100 if total_images > 0 else 0
    TIR = correct_identifications / total_images * 100 if total_images > 0 else 0

    print(f"Total images processed: {total_images}")
    print(f"Correctly identified images: {correct_identifications}")
    print(f"False Rejection Rate (FRR): {FRR:.2f}%")
    print(f"False Acceptance Rate (FAR): {FAR:.2f}%")
    print(f"True Identification Rate (TIR): {TIR:.2f}%")

    return FRR, FAR, TIR

# Setup and testing
sr = prepare_super_resolution()
db_path = './database'
directory = 'test_6000'
FRR, FAR, TIR = test_images(directory, db_path, sr)
